In [ ]:
using COVIDIncidencePOMDPs
using POMCPOW
using POMDPModelTools
using Distributions
using POMDPSimulators
using POMDPs
using POMDPPolicies
using Random
#using ARDESPOT
using ParticleFilters
#using Statistics
#using UUIDs
#using MCTS
#using MCVI
#using Parameters
using POMCPOW
#using Roots
#using SpecialFunctions
#using QuadGK
#using LinearAlgebra



In [ ]:
p = PARAM_MAPPING["good"]

k, lam = get_weibull_params(p.MEAN_GEN_TIME, p.STD_GEN_TIME)
gen_func = discrete_weibull(k, lam, p.TOTAL_TIME)
base_R0 = calc_R0(k, lam, p.EXP_GROWTH, p.TOTAL_TIME)
initial_incidences = ones(Float64, p.TOTAL_TIME) * p.INFECTION_SEED
#initial_belief = InitialBeliefDistribution(initial_incidences, p.NUM_LEVELS)
initial_belief = InitialBeliefDistribution(initial_incidences, 10.0, p.NUM_LEVELS)

covid_pomdp = CovidPOMDP(base_R0, gen_func, initial_incidences, p)

solver = POMCPOWSolver(max_depth=20, tree_queries=1,
    k_observation=6.0, alpha_observation=1/20., enable_action_pw=false)#,
    #estimate_value=RolloutEstimator(FixedSolver()))

planner = solve(solver, covid_pomdp)
updater = SIRParticleFilter(covid_pomdp, 100)

#stepper = stepthrough(covid_pomdp, planner, updator, initial_belief, "b,s,sp,a,o,r", max_steps=200)

#sim = RolloutSimulator()
sim = HistoryRecorder(max_steps=100)
stepper = simulate(sim, covid_pomdp, planner, updater, initial_belief)

for (s, b, a, r, sp, o) in stepper
    #@show s, b, a, r, sp, o
    @show r
end